In [29]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit.circuit import QuantumCircuit
from copy import  deepcopy
from qiskit.circuit import Parameter,ParameterVector
import sys
sys.path.append('..')
from AdaptVQE_.My_adapt import MyG_AdaotVQE,MyAdaptVQE
# Define Molecule
molecule = MoleculeInfo(
    ["Li", "H"], [(0.0, 0.0, 0.0), (0.8*1, 0.0, 0.0)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)

problem = driver.run()
converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True)
transformer = FreezeCoreTransformer(
    freeze_core=True,  # how many electrons we have in our active space
    # how many orbitals we have in our active space
    remove_orbitals=[-3, -2]
)
#problem = transformer.transform(problem=problem)

In [36]:
H = converter.convert(problem.hamiltonian.second_q_op())
len(H.to_pauli_op().oplist[0].primitive.to_label())

12

In [44]:
paulis = H.to_pauli_op().oplist
paulis_dict = {}
paulis[]

[PauliOp(Pauli('IIIIIIIIIIII'), coeff=-5.773759386278723),
 PauliOp(Pauli('IIIIIIIIIIIZ'), coeff=1.0733801096754303),
 PauliOp(Pauli('IIIIIIIIIIYY'), coeff=0.028279930077996673),
 PauliOp(Pauli('IIIIIIIIIIXX'), coeff=0.028279930077996673),
 PauliOp(Pauli('IIIIIIIIIYZY'), coeff=0.022827880598626914),
 PauliOp(Pauli('IIIIIIIIIXZX'), coeff=0.022827880598626914),
 PauliOp(Pauli('IIIIIIYZZZZY'), coeff=0.0292359169713985),
 PauliOp(Pauli('IIIIIIXZZZZX'), coeff=0.0292359169713985),
 PauliOp(Pauli('IIIIIIIIIIZI'), coeff=-0.08874775717026108),
 PauliOp(Pauli('IIIIIIIIIYYI'), coeff=-0.007918678685051223),
 PauliOp(Pauli('IIIIIIIIIXXI'), coeff=-0.007918678685051223),
 PauliOp(Pauli('IIIIIIYZZZYI'), coeff=-0.018169196575976744),
 PauliOp(Pauli('IIIIIIXZZZXI'), coeff=-0.018169196575976744),
 PauliOp(Pauli('IIIIIIIIIZII'), coeff=-0.18109182434550247),
 PauliOp(Pauli('IIIIIIYZZYII'), coeff=0.012579387004280837),
 PauliOp(Pauli('IIIIIIXZZXII'), coeff=0.012579387004280837),
 PauliOp(Pauli('IIIIIIIIZIII

In [38]:
def pauliOperator2Dict(pauliOperator):
    """
    Converts pauli operator to dict: pauli string as keys and coefficient as value
    Example:
    {I: 0.4, X: 0.6, Z: 0.1, Y: 0.5}.
    
    Args:
        pauliOperator (PauliSumOp): Pauli operator
    Returns:
        dict: Dictionary containing pauli strings as key and coefficient as value
    """
    paulis = pauliOperator.to_pauli_op().oplist
    paulis_dict = {}

    for x in paulis:
        if x.coeff == 0.0:
            continue
        label = x.primitive.to_label()
        coeff = x.coeff
        paulis_dict[label] = coeff

    return paulis_dict

In [39]:
pauliOperator2Dict(H)

{'IIIIIIIIIIII': -5.773759386278723,
 'IIIIIIIIIIIZ': 1.0733801096754303,
 'IIIIIIIIIIYY': 0.028279930077996673,
 'IIIIIIIIIIXX': 0.028279930077996673,
 'IIIIIIIIIYZY': 0.022827880598626914,
 'IIIIIIIIIXZX': 0.022827880598626914,
 'IIIIIIYZZZZY': 0.0292359169713985,
 'IIIIIIXZZZZX': 0.0292359169713985,
 'IIIIIIIIIIZI': -0.08874775717026108,
 'IIIIIIIIIYYI': -0.007918678685051223,
 'IIIIIIIIIXXI': -0.007918678685051223,
 'IIIIIIYZZZYI': -0.018169196575976744,
 'IIIIIIXZZZXI': -0.018169196575976744,
 'IIIIIIIIIZII': -0.18109182434550247,
 'IIIIIIYZZYII': 0.012579387004280837,
 'IIIIIIXZZXII': 0.012579387004280837,
 'IIIIIIIIZIII': -0.21226730641042774,
 'IIIIIIIZIIII': -0.21226730641042782,
 'IIIIIIZIIIII': -0.38221460065005225,
 'IIIIIZIIIIII': 1.073380109675431,
 'IIIIYYIIIIII': 0.028279930077996708,
 'IIIIXXIIIIII': 0.028279930077996708,
 'IIIYZYIIIIII': 0.02282788059862694,
 'IIIXZXIIIIII': 0.02282788059862694,
 'YZZZZYIIIIII': 0.029235916971398532,
 'XZZZZXIIIIII': 0.029235916971398

In [5]:
from qiskit_nature.circuit.library import UCCSD
uccsd = UCCSD(num_particles=problem.num_particles,num_spin_orbitals=problem.num_spatial_orbitals,
              qubit_converter=converter)
uccsd.excitation_ops()

[FermionicOp([('+I-III', 1j), ('-I+III', 1j)], register_length=6, display_format='dense'),
 FermionicOp([('I+-III', 1j), ('I-+III', 1j)], register_length=6, display_format='dense'),
 FermionicOp([('III+I-', 1j), ('III-I+', 1j)], register_length=6, display_format='dense'),
 FermionicOp([('IIII+-', 1j), ('IIII-+', 1j)], register_length=6, display_format='dense'),
 FermionicOp([('+I-+I-', 1j), ('-I+-I+', (-0-1j))], register_length=6, display_format='dense'),
 FermionicOp([('+I-I+-', 1j), ('-I+I-+', (-0-1j))], register_length=6, display_format='dense'),
 FermionicOp([('I+-+I-', 1j), ('I-+-I+', (-0-1j))], register_length=6, display_format='dense'),
 FermionicOp([('I+-I+-', 1j), ('I-+I-+', (-0-1j))], register_length=6, display_format='dense')]

In [22]:
Paulisum = [converter.convert(i) for i in uccsd.excitation_ops()]
Paulisum[0].primitive[0]

SparsePauliOp(['IIIXZY'],
              coeffs=[0.5+0.j])

In [26]:
Paulisum[0].primitive.to_list()[0][0]

'IIIXZY'

In [28]:
from openvqe.ucc_family import get_energy_ucc

In [49]:
from qiskit.opflow import X,Y,Z,I
from qiskit.visualization import array_to_latex

In [59]:
(X^Y^Y^Y).to_matrix()
array_to_latex(1j*(X^Y^Y^Y).to_matrix())

<IPython.core.display.Latex object>

In [58]:
matrix = (Y^Y^Y^X).to_matrix()
array_to_latex(1j*(Y^Y^Y^X).to_matrix())

<IPython.core.display.Latex object>

In [57]:
(Y^Y^Y^X)

PauliOp(Pauli('YYYX'), coeff=1.0)